In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import pickle
from utils import lla2enu, ecef2enu, get_google_data, find_nearest_time, find_nearest_pos, quat2euler

## GPS and IMU measurements

In [ ]:
# read IMU data
imu_path = "../data/csv_data/imu.csv"
imu = pd.read_csv(imu_path)
imu['rosbagTimestamp'] = imu['secs']*int(1e9) + imu['nsecs']
imu = imu.drop(['header', 'seq', 'stamp', 'secs', 'nsecs'], axis=1)

imu.rename(columns={'frame_id':'has_imu', 'rosbagTimestamp':'time'}, inplace=True)
imu['has_imu'] = True

imu.rename(columns={f'{i}': f'imu_{i}' for i in ['ang_vel_x', 'ang_vel_y', 'ang_vel_z', 'lin_acc_x', 'lin_acc_y', 'lin_acc_z']}, inplace=True)

imu

In [ ]:
# read GPS data
gps_path = '../data/csv_data/gps.csv'
gps = pd.read_csv(gps_path)
gps.drop(['header', 'seq', 'stamp', 'secs', 'nsecs', 'status', 'status.1', 'service', 'position_covariance', 'position_covariance_type'], axis=1, inplace=True)
gps.rename(columns={'frame_id':'has_gps', 'rosbagTimestamp':'time'}, inplace=True)
gps['has_gps'] = True

# convert to local ECEF (linearized around ORIGIN_LAT, ORIGIN_LON, ORIGIN_ALT defined in utils.py)
gps[['gps_pos_x', 'gps_pos_y', 'gps_pos_z']] = gps.apply(lambda row: lla2enu(row.latitude, row.longitude, row.altitude), axis=1, result_type='expand')
gps.drop(['latitude', 'longitude', 'altitude'], axis=1, inplace=True)

gps

In [ ]:
# merge GPS and IMU
meas = pd.concat([gps, imu]).sort_values('time')
meas = meas.groupby('time').aggregate(max).reset_index()

meas

## Google maps

In [ ]:
google = get_google_data('../data/map_to_gpx.gpx', alt=np.zeros(500000)).T
road = pd.DataFrame({'road_lat':google[0], 'road_lon':google[1], 'road_alt':google[2]})
road[['road_x', 'road_y', 'road_z']] = road.apply(lambda row: ecef2enu(row.road_lat, row.road_lon, row.road_alt), axis=1, result_type='expand')
road.drop(['road_lat', 'road_lon', 'road_alt'], axis=1, inplace=True)
road.diff().describe()

## Camera data

In [ ]:
with open("../data/deviation_from_center", "rb") as fp:
    dev_from_center = pickle.load(fp)
    dev_from_center = pd.DataFrame({'delta':dev_from_center})
    dev_from_center = pd.DataFrame({'delta':dev_from_center.apply(lambda row: row.delta * (3.05/3.5), axis=1)}) # BUG: Quick fix for wrong LW in utils.py

with open("../data/img_data_dir", "rb") as fp:
    cam_timestamps = pickle.load(fp)
    cam_timestamps = [int(cam_timestamp.split('.')[0])*1000 for cam_timestamp in cam_timestamps]
    cam_timestamps = pd.DataFrame({'cam_timestamps':cam_timestamps})

cam_times = find_nearest_time(gps['time'], cam_timestamps['cam_timestamps'])
cam_times['has_cam'] = True
cam = cam_times.join(dev_from_center).sort_values('time')

cam

In [ ]:
cam_pos = gps.merge(cam, on='time')[['time', 'gps_pos_x', 'gps_pos_y']]
centerlane = find_nearest_pos(road['road_x'], road['road_y'], cam_pos['gps_pos_x'], cam_pos['gps_pos_y'])
cam_pos = cam_pos.join(centerlane).drop(['gps_pos_x', 'gps_pos_y'], axis=1)
cam = cam.merge(cam_pos, on='time')

cam

In [ ]:
# Add current lane, LANEWIDTH, number of lanes

# Hand-labeled from images
idxs  = np.array([0, 242, 250, 761, 794, 838, 872, 1098, 1130, 1724, 1751, 2215, 2291, 2943, 3509, 3607, 4877]) # frame index where a lane change happens
lanes = np.array([     1, 0.5,   0,   0,   1,   0,    0,    1,    0,  0.5,    1,    0,    1,    0,    1,    2]) # lane the car is in, rightmost=0
lens  = np.diff(idxs)
curr_lane = np.empty(0)
for le, lane in zip(lens, lanes):
    curr_lane = np.hstack((curr_lane, np.ones(le)*lane))
cam['curr_lane'] = curr_lane
cam['LANEWIDTH'] = 3.05
cam['n_lanes']   = 3

cam

In [ ]:
# Add camera data to meas
meas = meas.join(cam.set_index('time'), on='time')
meas = meas.fillna(value={"has_cam": False})
meas['has_cam'] = meas.apply(lambda row: False if np.isnan(row.delta) else True, axis=1)

meas

## Add heading data

In [10]:
# read pose_raw data
pose_path = '../data/csv_data/pose_raw.csv'
pose = pd.read_csv(pose_path)
pose.drop(['header', 'seq', 'stamp', 'secs', 'nsecs', 'frame_id', 'pose', 'position','orientation'], axis=1, inplace=True)
pose.rename(columns={'rosbagTimestamp':'time'}, inplace=True)
pose = pose.drop(['x', 'y', 'z'], axis=1)
pose.rename(columns={'x.1':'qx', 'y.1':'qy','z.1':'qz', 'w':'qw'}, inplace=True)
pose[['gps_ang_theta', 'gps_ang_phi', 'gps_ang_varphi']] = pose.apply(lambda row: quat2euler((row.qx, row.qy, row.qz, row.qw)), axis=1, result_type='expand')
pose = pose.drop(['qx', 'qy', 'qz', 'qw'], axis=1)
pose['has_angle'] = True

pose

In [ ]:
# Add heading data to meas
meas = meas.join(pose.set_index('time'), on='time')
meas = meas.fillna(value={'has_angle': False})

meas

In [12]:
meas.to_csv('../data/measurements.csv')

## GPS ground truth

In [13]:
gt_path = '../data/csv_data/pose_ground_truth.csv'
gt = pd.read_csv(gt_path)[['rosbagTimestamp', 'x', 'y', 'z']]
gt.rename(columns={'rosbagTimestamp':'time', 'x':'x_gt', 'y':'y_gt', 'z':'z_gt'}, inplace=True)
gt.to_csv('../data/ground_truth.csv')